In [1]:
import tensorflow as tf
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '15'
import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
import re
import string
from keras.metrics import CategoricalAccuracy, F1Score

2024-05-10 12:09:18.769550: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-10 12:09:18.796727: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-10 12:09:18.796754: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-10 12:09:18.797408: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-10 12:09:18.802093: I tensorflow/core/platform/cpu_feature_guar

In [2]:
base_dir = '6-persian-topics'

In [3]:
num = os.listdir(base_dir)

In [4]:
data_features = []
data_targets = []
for n in num:
    class_path = os.path.join(base_dir , n)
    file_path = os.listdir(class_path)
    for p in file_path:
        matn = os.path.join(class_path , p)
        file = open(matn , 'r')
        file_r = file.read()
        if len(file_r) < 20:
            os.remove(matn)
        else:
            data_features.append(matn)
            data_targets.append(num.index(n))
        file.close()

In [5]:
train_f , val_f , train_t , val_t = train_test_split(data_features , data_targets , random_state=42 , test_size= 0.2)


In [6]:
len(data_features)

37253

In [7]:
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  stripped_ye = tf.strings.regex_replace(stripped_html, 'ي', 'ی')
  stripped_ke = tf.strings.regex_replace(stripped_ye, 'ك', 'ک')
  stripped_alef = tf.strings.regex_replace(stripped_ke, 'آ', 'ا')
  stripped_english = tf.strings.regex_replace(stripped_alef, '[a-zA-Z]', ' ')
  return tf.strings.regex_replace(stripped_alef,
                                  '[%s]' % re.escape(string.punctuation),'')

In [8]:
max_features = 10000
sequence_length = 250

vectorize_layer = layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)


2024-05-10 12:09:23.471842: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-10 12:09:23.492137: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-10 12:09:23.492313: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [9]:
AUTOTUNE = tf.data.AUTOTUNE

def load_text(path, lbl):
    lbl = tf.one_hot(lbl, depth=6, dtype='float32')
    text = tf.io.read_file(path)
    return text, lbl

def vectorize_text(text, lbl):
    text = vectorize_layer(text)
    return text, lbl

train_ds = tf.data.Dataset.from_tensor_slices((train_f, train_t))
val_ds = tf.data.Dataset.from_tensor_slices((val_f, val_t))

train_ds = train_ds.map(load_text)
vectorize_layer.adapt(train_ds.map(lambda text, label: text))
train_ds = train_ds.map(vectorize_text)

val_ds = val_ds.map(load_text)
val_ds = val_ds.map(vectorize_text)

train_ds = train_ds.shuffle(1000).batch(128)
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().batch(128).prefetch(buffer_size=AUTOTUNE)


In [10]:
embedding_dim = 16

In [11]:
model = tf.keras.Sequential([
  layers.Embedding(max_features, embedding_dim),
  layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim)),
  layers.Dense(100),
  layers.Dropout(0.2),
  layers.Dense(6)])

model.summary()


2024-05-10 12:10:18.012514: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 16)          160000    
                                                                 
 bidirectional (Bidirection  (None, 32)                4224      
 al)                                                             
                                                                 
 dense (Dense)               (None, 100)               3300      
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 6)                 606       
                                                                 
Total params: 168130 (656.76 KB)
Trainable params: 168130 (656.76 KB)
Non-trainable params: 0 (0.00 Byte)
________________

In [12]:
metrics = metrics = [
    CategoricalAccuracy(name='accuracy'),
    F1Score(name='f1-score')
]


In [13]:
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(0.01),
              metrics=metrics)

In [14]:
train_iterator = train_ds.as_numpy_iterator()

class_counts = {}
class_indices = {}


class_index = 0
for data, labels in train_iterator:
    for label in labels:
        label_tuple = tuple(label)
        if label_tuple not in class_counts:
            class_counts[label_tuple] = 1
            class_indices[label_tuple] = class_index
            class_index += 1
        else:
            class_counts[label_tuple] += 1

total_samples = sum(class_counts.values())


class_weights = {
    class_indices[label]: total_samples / count
    for label, count in class_counts.items()
}



In [15]:
history = model.fit(train_ds, epochs=10,
                    validation_data=val_ds,
                    validation_steps=30,
                    class_weight=class_weights)

Epoch 1/10


2024-05-10 12:10:26.881411: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-05-10 12:10:27.607521: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f2f10afd0d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-05-10 12:10:27.607548: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce MX330, Compute Capability 6.1
2024-05-10 12:10:27.611869: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1715357427.676116    8112 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


233/233 [==============================] - 23s 88ms/step - loss: 5.7700 - accuracy: 0.8908 - f1-score: 0.5673 - val_loss: 0.1887 - val_accuracy: 0.9529 - val_f1-score: 0.7158
Epoch 2/10
  7/233 [..............................] - ETA: 5s - loss: 1.8822 - accuracy: 0.9598 - f1-score: 0.7350

2024-05-10 12:10:48.588463: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


233/233 [==============================] - 6s 26ms/step - loss: 1.1940 - accuracy: 0.9719 - f1-score: 0.7652 - val_loss: 0.1562 - val_accuracy: 0.9594 - val_f1-score: 0.7502
Epoch 3/10
  6/233 [..............................] - ETA: 5s - loss: 0.7537 - accuracy: 0.9792 - f1-score: 0.8048

...

2024-05-10 12:11:31.231323: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


233/233 [==============================] - 6s 26ms/step - loss: 0.0901 - accuracy: 0.9970 - f1-score: 0.9786 - val_loss: 0.1362 - val_accuracy: 0.9760 - val_f1-score: 0.8823
Epoch 10/10
  7/233 [..............................] - ETA: 5s - loss: 0.0669 - accuracy: 0.9978 - f1-score: 0.9874

2024-05-10 12:11:37.347461: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


233/233 [==============================] - 6s 26ms/step - loss: 0.0673 - accuracy: 0.9979 - f1-score: 0.9873 - val_loss: 0.1379 - val_accuracy: 0.9758 - val_f1-score: 0.8798


2024-05-10 12:11:43.430508: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [16]:
result = model.evaluate(val_ds)
print('Evaluation result:', result)

59/59 [==============================] - 1s 10ms/step - loss: 0.1431 - accuracy: 0.9768 - f1-score: 0.8921
Evaluation result: [0.1430804431438446, 0.9767816662788391, array([0.9258778 , 0.72868216, 0.9920608 , 0.98771495, 0.7539683 ,
       0.96449023], dtype=float32)]


In [17]:
predictions = model.predict(val_ds)
predicted_classes = tf.math.argmax(predictions, axis=-1)

59/59 [==============================] - 1s 9ms/step


In [18]:
from sklearn.metrics import classification_report
report = classification_report(val_t, predicted_classes)

print(report)

              precision    recall  f1-score   support

           0       0.95      0.91      0.93       393
           1       0.84      0.64      0.73        73
           2       0.99      0.99      0.99      4339
           3       0.99      0.99      0.99      1224
           4       0.68      0.84      0.75       113
           5       0.96      0.96      0.96      1309

    accuracy                           0.98      7451
   macro avg       0.90      0.89      0.89      7451
weighted avg       0.98      0.98      0.98      7451

